<img src="Immagine1.png">

Import delle librerie

In [1]:
import pandas as pd
import re, math
import datetime as dt 
import numpy as np
import collections
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
pd.options.mode.chained_assignment = None 

Import dei files

In [2]:
times= pd.read_csv(r'C:\Users\valen\OneDrive\Documenti\Data science\Foundations of computer science\ProgettoCS\timesData.csv')
shanghai= pd.read_csv(r'C:\Users\valen\OneDrive\Documenti\Data science\Foundations of computer science\ProgettoCS\shanghaiData.csv')
attainment= pd.read_csv(r'C:\Users\valen\OneDrive\Documenti\Data science\Foundations of computer science\ProgettoCS\educational_attainment_supplementary_data.csv')
school_country= pd.read_csv(r'C:\Users\valen\OneDrive\Documenti\Data science\Foundations of computer science\ProgettoCS\school_and_country_table.csv')
expenditure= pd.read_csv(r'C:\Users\valen\OneDrive\Documenti\Data science\Foundations of computer science\ProgettoCS\education_expenditure_supplementary_data.csv',sep=',', lineterminator='\r')
cwur= pd.read_csv(r'C:\Users\valen\OneDrive\Documenti\Data science\Foundations of computer science\ProgettoCS\cwurData.csv')

Esplorazione dei dataset

In [3]:
times.head(2)

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011


check dei valori nulli per l'attributo university_name

In [4]:
times[times['university_name'].isnull()==True]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year


In [5]:
shanghai.head(2)

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005


check dei valori nulli per l'attributo university_name

In [6]:
shanghai[shanghai['university_name'].isnull()==True]

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
3896,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013


Rimozione record con valori nulli e reset dell'index

In [7]:
shanghai.drop([3896],inplace = True)

In [8]:
shanghai.reset_index(drop=True,inplace=True)

In [9]:
cwur.head(2)

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012


In [10]:
cwur.rename(columns={'institution':'university_name'},inplace=True)

check dei valori nulli per l'attributo university_name

In [11]:
cwur[cwur['university_name'].isnull()==True]

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year


**Valutazione similarità tra i nomi delle università nei tre dataset**

Rimozione dei caratteri speciali nel nome delle univeristà e lowercase 

In [12]:
shanghai['university_name'] = shanghai['university_name'].str.replace('[^A-Za-z0-9]', ' ').str.replace('  ', ' ').str.replace('   ', ' ').str.lower().str.strip()
times['university_name'] = times['university_name'].str.replace('[^A-Za-z0-9]', ' ').str.replace('  ', ' ').str.replace('   ', ' ').str.lower().str.strip()
cwur['university_name'] = cwur['university_name'].str.replace('[^A-Za-z0-9]', ' ').str.replace('  ', ' ').str.replace('   ', ' ').str.lower().str.strip()

Creazione df con nomi unici delle università sui tre dataset ranking

In [13]:
times_unique_df = pd.DataFrame(times['university_name'].unique(),columns=['university_name'])
shanghai_unique_df = pd.DataFrame(shanghai['university_name'].unique(),columns=['university_name'])
cwur_unique_df = pd.DataFrame(cwur['university_name'].unique(),columns=['university_name'])
List = [times_unique_df,shanghai_unique_df, cwur_unique_df]  # Lista dei df 
df = pd.concat(List)
univ_unique = pd.DataFrame(df['university_name'].unique(),columns=['university_name'])

In [14]:
univ_unique

,university_name
0,harvard university
1,california institute of technology
2,massachusetts institute of technology
3,stanford university
4,princeton university
...,...
1414,ulsan national institute of science and techno...
1415,shenzhen university
1416,tianjin medical university
1417,henan normal university


Costruzione doc-term matrix

In [15]:
cv = CountVectorizer() 
count_matrix = cv.fit_transform(univ_unique['university_name']) 

In [16]:
count_matrix.shape

(1419, 1375)

In [17]:
doc_term_matrix = count_matrix.todense()
df_matrix = pd.DataFrame(doc_term_matrix, 
                  columns=cv.get_feature_names(), 
                  index=univ_unique['university_name'])
df_matrix

,11,13,aachen,aalborg,aalto,aarhus,abc,abdulaziz,abdullah,aberdeen,...,york,young,yuan,yvelines,zagreb,zaragoza,ze,zhejiang,zhengzhou,zurich
university_name,,,,,,,,,,,,,,,,,,,,,
harvard university,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
california institute of technology,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
massachusetts institute of technology,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
stanford university,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
princeton university,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ulsan national institute of science and technology,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
shenzhen university,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tianjin medical university,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Calcolo della misura di similarità tra i nomi delle università

In [18]:
cosine_sim = cosine_similarity(count_matrix)

In [19]:
df_cosine_sim= pd.DataFrame(cosine_sim, columns=univ_unique['university_name'], index= univ_unique['university_name'])

In [20]:
df_cosine_sim

university_name,harvard university,california institute of technology,massachusetts institute of technology,stanford university,princeton university,university of cambridge,university of oxford,university of california berkeley,imperial college london,yale university,...,hefei university of technology,university of lethbridge,shahid beheshti university of medical sciences,gachon university,harbin engineering university,ulsan national institute of science and technology,shenzhen university,tianjin medical university,henan normal university,southwest jiaotong university
university_name,,,,,,,,,,,,,,,,,,,,,
harvard university,1.000000,0.000000,0.000000,0.500000,0.500000,0.408248,0.408248,0.353553,0.0,0.500000,...,0.353553,0.408248,0.288675,0.500000,0.408248,0.000000,0.500000,0.408248,0.408248,0.408248
california institute of technology,0.000000,1.000000,0.750000,0.000000,0.000000,0.288675,0.288675,0.500000,0.0,0.000000,...,0.500000,0.288675,0.204124,0.000000,0.000000,0.566947,0.000000,0.000000,0.000000,0.000000
massachusetts institute of technology,0.000000,0.750000,1.000000,0.000000,0.000000,0.288675,0.288675,0.250000,0.0,0.000000,...,0.500000,0.288675,0.204124,0.000000,0.000000,0.566947,0.000000,0.000000,0.000000,0.000000
stanford university,0.500000,0.000000,0.000000,1.000000,0.500000,0.408248,0.408248,0.353553,0.0,0.500000,...,0.353553,0.408248,0.288675,0.500000,0.408248,0.000000,0.500000,0.408248,0.408248,0.408248
princeton university,0.500000,0.000000,0.000000,0.500000,1.000000,0.408248,0.408248,0.353553,0.0,0.500000,...,0.353553,0.408248,0.288675,0.500000,0.408248,0.000000,0.500000,0.408248,0.408248,0.408248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ulsan national institute of science and technology,0.000000,0.566947,0.566947,0.000000,0.000000,0.218218,0.218218,0.188982,0.0,0.000000,...,0.377964,0.218218,0.154303,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
shenzhen university,0.500000,0.000000,0.000000,0.500000,0.500000,0.408248,0.408248,0.353553,0.0,0.500000,...,0.353553,0.408248,0.288675,0.500000,0.408248,0.000000,1.000000,0.408248,0.408248,0.408248
tianjin medical university,0.408248,0.000000,0.000000,0.408248,0.408248,0.333333,0.333333,0.288675,0.0,0.408248,...,0.288675,0.333333,0.471405,0.408248,0.333333,0.000000,0.408248,1.000000,0.333333,0.333333


Selezione delle università con nomi aventi una similarità elevata (compresa tra 0.85 e 0.99, con esclusione dei match diretti)

In [21]:
df_similar_uni=pd.DataFrame(df_cosine_sim.where(np.triu((df_cosine_sim.values >= 0.85) & (df_cosine_sim.values <= 0.99))).stack().sort_values(ascending=False))
df_similar_uni.rename(columns={0: 'cosine_sim'}, inplace=True)
df_similar_uni.index.set_names(["uni1", "uni2"], inplace=True)
df_similar_uni.reset_index(inplace=True)
pd.set_option('display.max_rows', 300)
df_similar_uni=df_similar_uni.sort_values('cosine_sim', ascending=False).drop_duplicates('uni1').sort_index()
df_similar_uni

,uni1,uni2,cosine_sim
0,state university of new york at albany,university at albany state university of new york,0.956183
1,university of medicine and dentistry of new je...,university of medicine and dentistry new jersey,0.956183
2,swiss federal institute of technology of lausanne,swiss federal institute of technology lausanne,0.952579
3,university of science and technology of china,university of electronic science and technolog...,0.948683
4,the university of texas health science center ...,university of texas health science center at s...,0.948683
5,paris dauphine university,paris dauphine university paris 9,0.942809
6,the university of texas health science center ...,university of texas health science center at h...,0.942809
7,indiana university purdue university at indian...,indiana university purdue university indianapolis,0.935414
8,the university of texas medical branch at galv...,university of texas medical branch at galveston,0.935414
9,hong kong university of science and technology,the hong kong university of science and techno...,0.935414


Costruzione dizionario per i risultati contenuti nel df mostrato al punto precedente

In [22]:
univ_sim_dict = dict(zip(df_similar_uni.uni1, df_similar_uni.uni2))

Prima della modifica il ds times aveva 818 nomi unici di università

In [23]:
times["university_name"].nunique()

818

Replacement dei nomi delle università contenuti nei tre dataset dei ranking con quelli che trovano un match nel dizionario costruito al punto precedente. Se non esiste alcun match, allora non viene modificato il nome dell'univeristà

In [24]:
times["university_name"].replace(univ_sim_dict, inplace=True)
shanghai["university_name"].replace(univ_sim_dict, inplace=True)
cwur["university_name"].replace(univ_sim_dict, inplace=True)

Dopo la sostituzione il ds times contiene 799 nomi unici di università

In [25]:
times["university_name"].nunique()

799

# **1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes**

In [26]:
times.year.describe()

count    2603.000000
mean     2014.075682
std         1.685733
min      2011.000000
25%      2013.000000
50%      2014.000000
75%      2016.000000
max      2016.000000
Name: year, dtype: float64

In [27]:
times_most_recent = times.loc[times.groupby('university_name')['year'].idxmax()]
times_most_recent.head(3)

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2003,201-250,aalborg university,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2056,251-300,aalto university,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
1908,=106,aarhus university,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016


In [28]:
len(times_most_recent)

799

In [29]:
times_least_recent = times.loc[times.groupby('university_name')['year'].idxmin()]
times_least_recent.head(3)

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
501,301-350,aalborg university,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",15.9,15%,48 : 52,2012
502,301-350,aalto university,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012
166,167,aarhus university,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011


In [30]:
len(times_least_recent)

799

# **2. For each university, compute the improvement in income between the least recent and the most recent data points**

Check dei missing sul campo income

In [31]:
times.groupby('income').size()

income
-        218
100.0     68
24.2       5
24.5       2
24.6       1
        ... 
99.5       5
99.6       2
99.7       4
99.8      10
99.9      14
Length: 613, dtype: int64

Replacement di - con 00.00 e conversione del campo income in float

In [32]:
times_most_recent['income'].replace('-', '00.00', inplace=True)
times_least_recent['income'].replace('-', '00.00', inplace=True)

In [33]:
times_most_recent['income'] = times_most_recent['income'].astype(float)
times_least_recent['income'] = times_least_recent['income'].astype(float)

Merge dei df most recent e least recent 

In [34]:
merged_least_most=pd.merge(
            times_least_recent[['university_name', 'income', 'year']],
            times_most_recent[['university_name', 'income', 'year']],
                        on='university_name', suffixes=('_least_recent', '_most_recent')
            )
merged_least_most['income_diff']=merged_least_most['income_most_recent']-merged_least_most['income_least_recent']

Vengono eliminate quelle università per cui non può essere calcolata la differenza della variabile 'income' in quanto hanno 'income_least_recent' o 'income_most_recent' nullo.

In [35]:
merged_least_most = merged_least_most[(merged_least_most['income_least_recent'] != 0.0) & (merged_least_most['income_most_recent'] != 0.0)]

In [36]:
merged_least_most.dropna(subset=['income_least_recent','income_most_recent'], inplace=True)

In [37]:
merged_least_most.sort_values(by='university_name')

,university_name,income_least_recent,year_least_recent,income_most_recent,year_most_recent,income_diff
0,aalborg university,36.4,2012,43.7,2016,7.3
1,aalto university,61.9,2012,61.6,2016,-0.3
2,aarhus university,61.5,2011,68.3,2016,6.8
3,aberystwyth university,35.5,2012,31.3,2016,-4.2
4,adam mickiewicz university,28.7,2016,28.7,2016,0.0
...,...,...,...,...,...,...
794,yokohama national university,40.4,2016,40.4,2016,0.0
795,yonsei university,40.4,2011,75.9,2016,35.5
796,york university,41.7,2012,33.2,2016,-8.5
797,yuan ze university,29.2,2012,39.8,2016,10.6


# **3. Find the university with the largest increase computed in the previous point**


In [38]:
univ_max_impr = merged_least_most.income_diff.idxmax()

In [39]:
merged_least_most[merged_least_most.index == univ_max_impr]

,university_name,income_least_recent,year_least_recent,income_most_recent,year_most_recent,income_diff
476,tu dresden,31.9,2012,99.7,2016,67.8


# **4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range**

I ranking sono presenti nei ds di times, cwur e shanghai

In [40]:
shanghai.year.describe()

count    4896.000000
mean     2009.657884
std         3.197546
min      2005.000000
25%      2007.000000
50%      2009.000000
75%      2012.000000
max      2015.000000
Name: year, dtype: float64

world_rank espresso in range nel ds shanghai e total score non valorizzato

In [41]:
shanghai[shanghai['world_rank'].str.contains("-")]

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
100,101-152,aarhus university,2,NaN,15.4,19.3,7.9,22.3,41.6,22.4,2005
101,101-152,arizona state university tempe,54-71,NaN,0.0,14.4,20.8,26.3,41.9,17.5,2005
102,101-152,baylor college of medicine,54-71,NaN,0.0,0.0,17.6,34.5,44.0,24.9,2005
103,101-152,catholic university of leuven,1-4,NaN,0.0,0.0,19.2,16.0,48.7,23.1,2005
104,101-152,catholic university of louvain,1-4,NaN,14.0,13.9,13.6,8.3,44.7,26.9,2005
...,...,...,...,...,...,...,...,...,...,...,...
4891,401-500,university of trieste,11-20,NaN,0.0,0.0,5.0,10.9,25.1,20.1,2015
4892,401-500,university of zaragoza,9-13,NaN,0.0,0.0,7.6,5.1,33.3,13.1,2015
4893,401-500,utah state university,126-146,NaN,13.6,0.0,3.6,10.8,25.1,15.5,2015
4894,401-500,vienna university of technology,4-6,NaN,0.0,0.0,0.0,12.2,28.8,22.9,2015


In [42]:
shanghai = shanghai.fillna(0)

Estrazione dei ranking più recenti per i ds shanghai e cwur. Per il ds times è già stato creato il df most_recent al punto 2

In [43]:
shanghai_most_recent=shanghai.loc[shanghai.groupby('university_name')['year'].idxmax()]
cwur_most_recent=cwur.loc[cwur.groupby('university_name')['year'].idxmax()]

Nella colonna 'world_rank' del dataset 'times' alcuni valori contengono il carattere stringa '=' che precede il valore indicativo del ranking. Viene quindi eliminato il carattere stringa '=' qualora sia presente nei dati.

In [44]:
def rank(df):
    df['world_rank']=df['world_rank'].str.replace('=','')
    df['world_rank_min']=df['world_rank'].str.extract(r'(^\d+)').astype(int)
    df['world_rank_max']=df['world_rank'].str.extract(r'(\d+$)').astype(int)
    df['world_rank_avg']=(df['world_rank_min']+df['world_rank_max'])/2
    return df

In [45]:
rank(times_most_recent)
rank(shanghai_most_recent)

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,world_rank_min,world_rank_max,world_rank_avg
4696,301-400,aalborg university,5,0.0,0.0,0.0,11.2,4.6,30.4,16.8,2015,301,400,350.5
4796,401-500,aalto university,4-6,0.0,0.0,0.0,0.0,10.4,33.4,17.0,2015,401,500,450.5
4468,73,aarhus university,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,2015,73,73,73.0
4496,101-150,aix marseille university,5-6,0.0,13.6,0.0,15.2,20.3,50.1,27.6,2015,101,150,125.5
4797,401-500,aristotle university of thessaloniki,2,0.0,0.0,0.0,4.9,1.5,34.0,16.8,2015,401,500,450.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3512,401-500,yamaguchi university,17-23,0.0,0.0,0.0,16.1,2.8,20.0,13.2,2011,401,500,450.5
4694,201-300,yeshiva university,79-102,0.0,0.0,0.0,6.3,21.7,31.5,21.6,2015,201,300,250.5
4695,201-300,yonsei university,2-5,0.0,0.0,0.0,8.6,4.9,52.1,21.4,2015,201,300,250.5
4395,401-500,york university,19-21,0.0,0.0,0.0,0.0,10.1,30.6,12.4,2014,401,500,450.5


In [46]:
cwur_most_recent['world_rank_avg']=cwur_most_recent['world_rank']

In [47]:
total_most_recent=pd.concat([
                            times_most_recent, 
                            shanghai_most_recent, 
                            cwur_most_recent
                            ],sort=False)

In [48]:
total_most_recent.reset_index(drop=True, inplace=True)

In [49]:
total_min=total_most_recent.loc[total_most_recent.groupby('university_name')['world_rank_avg'].idxmin()][['university_name', 'world_rank_avg']]
total_min.rename(columns=({'world_rank_avg':'min'}), inplace=True)
total_max=total_most_recent.loc[total_most_recent.groupby('university_name')['world_rank_avg'].idxmax()][['university_name', 'world_rank_avg']]
total_max.rename(columns=({'world_rank_avg':'max'}), inplace=True)

In [50]:
df4=pd.merge(total_min, total_max, on='university_name')
df4['diff']=df4['max']-df4['min']
df4.head()

,university_name,min,max,diff
0,aalborg university,225.5,565.0,339.5
1,aalto university,275.5,450.5,175.0
2,aarhus university,73.0,122.0,49.0
3,aberystwyth university,325.5,814.0,488.5
4,adam mickiewicz university,700.5,700.5,0.0


Si verifica che l'esempio di differenza massima tra i rank fornito dal professore sia verificato.

In [51]:
df4[df4['university_name']=='aarhus university']

,university_name,min,max,diff
2,aarhus university,73.0,122.0,49.0


**Procedimento alternativo con il ricacolo dello score per times e shanghai**

In [52]:
shanghai['total_score'] = shanghai['total_score'].astype(float)

http://www.shanghairanking.com/ARWU-Methodology-2016.html --> metodologia per il calcolo dello score
aggiungo una colonna al df con i nuovi punteggi grezzi non ancora aggiustati rispetto al punteggio max per ciascun anno (il primo in classifica vale 100, gli altri punteggi vengono aggiustati in funzione di questo)

In [53]:
shanghai['new_total_score'] = (shanghai['alumni']*10/100+shanghai['award']*20/100+shanghai['hici']*20/100+
                               shanghai['ns']*20/100+shanghai['pub']*20/100+shanghai['pcp']*10/100)


In [54]:
shanghai.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,new_total_score
0,1,harvard university,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005,97.24
1,2,university of cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005,71.51
2,3,stanford university,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005,71.39
3,4,university of california berkeley,3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005,70.75
4,5,massachusetts institute of technology mit,4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005,68.18


Estrazione punteggi max per ciascun anno e creazione di un dizionario con il peso per ciascuno score max relativo ad ogni anno

In [55]:
top_scores_year = shanghai.groupby('year')['new_total_score'].max().reset_index()
top_scores_year['weight'] = 100/top_scores_year['new_total_score']
top_scores_year_dict = pd.Series(top_scores_year.weight.values,index=top_scores_year.year).to_dict()
top_scores_year_dict

{2005: 1.0283833813245578,
 2006: 1.0271158586688578,
 2007: 1.027749229188078,
 2008: 1.026588645929576,
 2009: 1.0258514567090684,
 2010: 1.031778786628147,
 2011: 1.0303967027305514,
 2012: 1.0297600659046442,
 2013: 1.0282776349614395,
 2014: 1.0253255408592228,
 2015: 1.0239606799098915}

Aggiunta della colonna contente il peso nel ds di shanghai

In [56]:
shanghai['weight_new_total_score'] = shanghai['year'].map(top_scores_year_dict)

shanghai[shanghai['university_name'] == 'aarhus university']

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,new_total_score,weight_new_total_score
100,101-152,aarhus university,2,0.0,15.4,19.3,7.9,22.3,41.6,22.4,2005,22.00,1.028383
601,102-150,aarhus university,2,0.0,14.8,18.9,7.7,23.8,42.7,24.6,2006,22.56,1.027116
1101,102-150,aarhus university,2,0.0,14.4,18.9,7.4,24.3,40.8,23.7,2007,22.09,1.027749
1603,93,aarhus university,2,24.5,13.7,18.9,7.3,24.5,48.2,27.0,2008,23.85,1.026589
2109,97,aarhus university,2,24.2,13.4,18.9,7.3,24.0,47.6,26.6,2009,23.56,1.025851
2611,98,aarhus university,2,24.2,13.1,18.9,7.2,23.5,48.4,25.4,2010,23.45,1.031779
3099,86,aarhus university,2,25.4,13.2,22.1,7.2,24.3,49.3,27.9,2011,24.69,1.030397
3599,86,aarhus university,2,25.5,12.1,22.8,7.2,23.9,50.8,26.6,2012,24.81,1.029760
3878,81,aarhus university,2.0,26.0,12.0,22.2,7.2,25.6,50.9,28.5,2013,25.23,1.028278
3969,74,aarhus university,2,27.4,11.8,22.1,12.3,26.7,51.9,29.8,2014,26.76,1.025326


Score aggiustato per il peso

In [57]:
shanghai['new_total_score'] = (shanghai['new_total_score']*shanghai['weight_new_total_score']).round(1)

In [58]:
shanghai

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,new_total_score,weight_new_total_score
0,1,harvard university,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005,100.0,1.028383
1,2,university of cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005,73.5,1.028383
2,3,stanford university,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005,73.4,1.028383
3,4,university of california berkeley,3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005,72.8,1.028383
4,5,massachusetts institute of technology mit,4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005,70.1,1.028383
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4891,401-500,university of trieste,11-20,0.0,0.0,0.0,5.0,10.9,25.1,20.1,2015,10.5,1.023961
4892,401-500,university of zaragoza,9-13,0.0,0.0,0.0,7.6,5.1,33.3,13.1,2015,10.8,1.023961
4893,401-500,utah state university,126-146,0.0,13.6,0.0,3.6,10.8,25.1,15.5,2015,11.1,1.023961
4894,401-500,vienna university of technology,4-6,0.0,0.0,0.0,0.0,12.2,28.8,22.9,2015,10.7,1.023961


In [59]:
shanghai = shanghai.groupby('year').apply(lambda x: x.sort_values('new_total_score', ascending= False )).reset_index(drop=True)
shanghai

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,new_total_score,weight_new_total_score
0,1,harvard university,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005,100.0,1.028383
1,2,university of cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005,73.5,1.028383
2,3,stanford university,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005,73.4,1.028383
3,4,university of california berkeley,3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005,72.8,1.028383
4,5,massachusetts institute of technology mit,4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005,70.1,1.028383
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4891,401-500,okayama university,13-18,0.0,0.0,0.0,3.6,9.0,28.3,11.7,2015,9.6,1.023961
4892,401-500,university of tokushima,13-18,0.0,13.6,0.0,5.1,8.0,19.7,14.7,2015,9.6,1.023961
4893,401-500,university of rennes 1,19-22,0.0,0.0,0.0,3.6,9.2,28.1,11.2,2015,9.5,1.023961
4894,401-500,nara institute of science and technology,13-18,0.0,0.0,0.0,13.3,13.5,13.9,11.4,2015,9.5,1.023961


Siccome nei records in cui sono presenti i world_rank non espressi in range sono presenti dei ranking ripetuti quando i total scores delle università sono uguali, creo un sottoindice per ciascun anno per tenere l'ordinamento originale ed assegno lo stesso posto in classifica se il punteggio del new_total_score è uguale per due università

In [60]:
shanghai['group_index_sha'] = shanghai.groupby('year').cumcount() + 1

In [61]:
shanghai['new_world_ranking_sha'] = shanghai['group_index_sha'] 

Per la creazione del nuovo ranking utilizziamo il new_total_rank 

In [62]:
for i in range(1, len(shanghai)):

    if shanghai.new_total_score.loc[i-1] != shanghai.new_total_score.loc[i]:
        shanghai['new_world_ranking_sha'].loc[i] = shanghai['group_index_sha'].loc[i]
        
    else:
        shanghai['new_world_ranking_sha'].loc[i] = shanghai.new_world_ranking_sha.loc[i-1]
    i = i+1

In [63]:
shanghai

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,new_total_score,weight_new_total_score,group_index_sha,new_world_ranking_sha
0,1,harvard university,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005,100.0,1.028383,1,1
1,2,university of cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005,73.5,1.028383,2,2
2,3,stanford university,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005,73.4,1.028383,3,3
3,4,university of california berkeley,3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005,72.8,1.028383,4,4
4,5,massachusetts institute of technology mit,4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005,70.1,1.028383,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4891,401-500,okayama university,13-18,0.0,0.0,0.0,3.6,9.0,28.3,11.7,2015,9.6,1.023961,496,492
4892,401-500,university of tokushima,13-18,0.0,13.6,0.0,5.1,8.0,19.7,14.7,2015,9.6,1.023961,497,492
4893,401-500,university of rennes 1,19-22,0.0,0.0,0.0,3.6,9.2,28.1,11.2,2015,9.5,1.023961,498,498
4894,401-500,nara institute of science and technology,13-18,0.0,0.0,0.0,13.3,13.5,13.9,11.4,2015,9.5,1.023961,499,498


Creazione del df con i dati più recenti per ciascuna università

In [64]:
shanghai_most_recent = shanghai.loc[shanghai.groupby('university_name')['year'].idxmax()].reset_index()

In [65]:
shanghai_most_recent

,index,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,new_total_score,weight_new_total_score,group_index_sha,new_world_ranking_sha
0,4792,301-400,aalborg university,5,0.0,0.0,0.0,11.2,4.6,30.4,16.8,2015,11.2,1.023961,397,392
1,4819,401-500,aalto university,4-6,0.0,0.0,0.0,0.0,10.4,33.4,17.0,2015,10.7,1.023961,424,424
2,4468,73,aarhus university,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,2015,27.3,1.023961,73,73
3,4527,101-150,aix marseille university,5-6,0.0,13.6,0.0,15.2,20.3,50.1,27.6,2015,21.7,1.023961,132,132
4,4868,401-500,aristotle university of thessaloniki,2,0.0,0.0,0.0,4.9,1.5,34.0,16.8,2015,10.0,1.023961,473,471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,3489,401-500,yamaguchi university,17-23,0.0,0.0,0.0,16.1,2.8,20.0,13.2,2011,9.4,1.030397,476,474
616,4660,201-300,yeshiva university,79-102,0.0,0.0,0.0,6.3,21.7,31.5,21.6,2015,14.4,1.023961,265,263
617,4634,201-300,yonsei university,2-5,0.0,0.0,0.0,8.6,4.9,52.1,21.4,2015,15.6,1.023961,239,237
618,4389,401-500,york university,19-21,0.0,0.0,0.0,0.0,10.1,30.6,12.4,2014,9.6,1.025326,494,491


In [66]:
shanghai_most_recent.year.describe()

count     620.000000
mean     2013.945161
std         2.375880
min      2005.000000
25%      2015.000000
50%      2015.000000
75%      2015.000000
max      2015.000000
Name: year, dtype: float64

In [67]:
shanghai_most_recent = shanghai_most_recent.sort_values(by =['year','new_world_ranking_sha'], ascending=True).reset_index(drop=True)
shanghai_most_recent[shanghai_most_recent['university_name']== 'aarhus university']

,index,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,new_total_score,weight_new_total_score,group_index_sha,new_world_ranking_sha
203,4468,73,aarhus university,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,2015,27.3,1.023961,73,73


Pulizia dei campi

In [68]:
times['income'].replace('-', '00.00', inplace=True)
times['international'].replace('-', '00.00', inplace=True)
times['world_rank'] = times['world_rank'].str.replace('=', '')
times['international_students'] = times['international_students'].str.replace('%', ' ')
times['income'] = times['income'].astype(float)
times['international'] = times['international'].astype(float)
times['international_students'] = times['international_students'].astype(float)

https://www.timeshighereducation.com/news/ranking-methodology-2016
La metodologia di calcolo del ranking è variata nel 2012
tuttavia, ci sono solo tre univ che hanno come dato più recente il 2011 quindi usare questo metodo consente di essere più precisi rispetto alla prima soluzione fornita con il calcolo del punto medio. 
Invece, non è consigliabile usare l'ordinamento dei ranking presente nel ds perchè per i ranking espressi in range l'ordine nel ds è alfabetico sul nome università
Come per il ds shanghai, se due univ hanno medesimo new total score, il new_world_rank è uguale

In [69]:
times['new_total_score'] = (times['teaching']*30/100+times['international']*7.5/100+times['research']*30/100+times['citations']*30/100+times['income']*2.5/100).round(1)

In [70]:
times = times.groupby('year').apply(lambda x: x.sort_values('new_total_score', ascending= False )).reset_index(drop=True)
times[times['year']==2012]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,new_total_score
200,1,california institute of technology,United States of America,95.7,56.0,98.2,99.9,97.0,94.8,"2,243",6.9,27.0,33 : 67,2012,94.8
201,2,stanford university,United States of America,94.8,57.2,98.9,99.8,63.8,93.9,"15,596",7.8,22.0,42 : 58,2012,93.9
202,2,harvard university,United States of America,95.8,67.5,97.4,99.8,35.9,93.9,"20,152",8.9,25.0,NaN,2012,93.9
203,4,university of oxford,United Kingdom,89.5,91.9,96.6,97.9,62.1,93.6,"19,919",11.6,34.0,46 : 54,2012,93.6
204,5,princeton university,United States of America,91.5,49.6,99.1,100.0,81.0,92.9,"7,929",8.4,27.0,45 : 55,2012,92.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,350-400,tokyo university of agriculture and technology,Japan,26.5,18.0,15.8,21.7,41.5,-,"5,865",14.4,5.0,27 : 73,2012,21.6
598,350-400,hiroshima university,Japan,27.7,18.8,16.3,20.1,39.4,-,"14,686",8.2,5.0,36 : 64,2012,21.6
599,350-400,new jersey institute of technology,United States of America,17.1,41.3,14.5,26.4,37.5,-,"10,646",26.2,17.0,25 : 75,2012,21.4
600,350-400,michigan technological university,United States of America,21.2,32.3,13.8,24.8,41.9,-,NaN,NaN,NaN,NaN,2012,21.4


In [71]:
times['group_index_times'] = times.groupby('year').cumcount() + 1

In [72]:
times['new_world_ranking_times'] = times['group_index_times'] 

In [73]:
for i in range(1, len(times)):
                
    if times.new_total_score.loc[i-1] != times.new_total_score.loc[i]:
        times['new_world_ranking_times'].loc[i] = times['group_index_times'].loc[i]
        
    else:
        times['new_world_ranking_times'].loc[i] = times.new_world_ranking_times.loc[i-1]
    i = i+1

In [74]:
times

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,new_total_score,group_index_times,new_world_ranking_times
0,1,harvard university,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25.0,NaN,2011,95.5,1,1
1,3,massachusetts institute of technology mit,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33.0,37 : 63,2011,95.1,2,2
2,2,california institute of technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27.0,33 : 67,2011,94.9,3,3
3,4,stanford university,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22.0,42 : 58,2011,92.5,4,4
4,5,princeton university,United States of America,90.9,70.3,95.4,99.9,0.0,94.2,"7,929",8.4,27.0,45 : 55,2011,91.1,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2598,601-800,southern federal university,Russian Federation,21.9,17.3,8.2,9.2,39.5,-,"27,556",9.0,3.0,56 : 44,2016,14.1,796,792
2599,601-800,v n karazin kharkiv national university,Ukraine,21.7,48.4,8.9,1.7,28.8,-,"14,410",9.7,22.0,53 : 47,2016,14.0,797,797
2600,601-800,federal university of bahia,Brazil,18.2,19.7,8.1,14.3,0.0,-,"39,861",17.9,1.0,47 : 53,2016,13.7,798,798
2601,601-800,university of bras lia,Brazil,21.9,24.1,7.4,10.4,0.0,-,"45,880",14.5,2.0,52 : 48,2016,13.7,799,798


In [75]:
times[times['world_rank'] == '1']

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,new_total_score,group_index_times,new_world_ranking_times
0,1,harvard university,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25.0,NaN,2011,95.5,1,1
200,1,california institute of technology,United States of America,95.7,56.0,98.2,99.9,97.0,94.8,"2,243",6.9,27.0,33 : 67,2012,94.8,1,1
602,1,california institute of technology,United States of America,96.3,59.8,99.4,99.7,95.6,95.5,"2,243",6.9,27.0,33 : 67,2013,95.5,1,1
1002,1,california institute of technology,United States of America,94.4,65.8,98.2,99.8,91.2,94.9,"2,243",6.9,27.0,33 : 67,2014,94.9,1,1
1402,1,california institute of technology,United States of America,92.2,67.0,98.1,99.7,89.1,94.3,"2,243",6.9,27.0,33 : 67,2015,94.3,1,1
1803,1,california institute of technology,United States of America,95.6,64.0,97.6,99.8,97.8,95.2,"2,243",6.9,27.0,33 : 67,2016,95.1,1,1


Creazione del df con i dati più recenti per ciascuna università

In [76]:
times_most_recent = times.loc[times.groupby('university_name')['year'].idxmax()].reset_index()

In [77]:
times_most_recent

,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,new_total_score,group_index_times,new_world_ranking_times
0,2051,201-250,aalborg university,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15.0,48 : 52,2016,44.6,249,247
1,2059,251-300,aalto university,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17.0,32 : 68,2016,44.2,257,255
2,1908,106,aarhus university,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14.0,54 : 46,2016,57.7,106,106
3,2143,301-350,aberystwyth university,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18.0,48 : 52,2016,38.5,341,341
4,2555,601-800,adam mickiewicz university,Poland,20.0,25.7,11.0,15.3,28.7,-,"40,633",15.6,1.0,71 : 29,2016,16.5,753,753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,2540,601-800,yokohama national university,Japan,20.1,23.3,16.0,13.5,40.4,-,"10,117",12.1,8.0,28 : 72,2016,17.6,738,737
795,2150,301-350,yonsei university,South Korea,37.2,41.5,34.1,39.4,75.9,-,"24,774",11.6,14.0,NaN,2016,38.2,348,347
796,2149,301-350,york university,Canada,30.2,58.2,30.8,49.1,33.2,-,"48,007",39.4,9.0,59 : 41,2016,38.2,347,347
797,2458,601-800,yuan ze university,Taiwan,16.2,17.7,18.3,28.6,39.8,-,"8,663",20.6,4.0,43 : 57,2016,21.3,656,653


In [78]:
cwur

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,harvard university,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,massachusetts institute of technology mit,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,stanford university,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,university of cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,california institute of technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,996,university of the algarve,Portugal,7,367,567,218,926,845,812,969.0,816,44.03,2015
2196,997,alexandria university,Egypt,4,236,566,218,997,908,645,981.0,871,44.03,2015
2197,998,federal university of cear,Brazil,18,367,549,218,830,823,812,975.0,824,44.03,2015
2198,999,university of a coru a,Spain,40,367,567,218,886,974,812,975.0,651,44.02,2015


In [79]:
cwur.rename(columns={'world_rank':'world_rank_cwur'}, inplace=True)

In [80]:
cwur_most_recent = cwur.loc[cwur.groupby('university_name')['year'].idxmax()].reset_index()

In [81]:
merged_rankings = shanghai_most_recent.merge(cwur_most_recent,on=['university_name'], how = 'outer').merge(times_most_recent,on=['university_name'], how='outer')[['university_name','new_world_ranking_sha','new_world_ranking_times','world_rank_cwur']]

In [82]:
merged_rankings

,university_name,new_world_ranking_sha,new_world_ranking_times,world_rank_cwur
0,college of france,151.0,NaN,NaN
1,northern arizona university,457.0,NaN,766.0
2,loyola university chicago,475.0,NaN,437.0
3,memorial university of newfoundland,484.0,553.0,611.0
4,university of wroclaw,484.0,NaN,NaN
...,...,...,...,...
1265,verona university,NaN,358.0,NaN
1266,wageningen university and research center,NaN,47.0,NaN
1267,west university of timi oara,NaN,785.0,NaN
1268,william mary,NaN,244.0,NaN


In [83]:
merged_rankings = merged_rankings.fillna(0.00)

In [84]:
merged_rankings_match = merged_rankings[((merged_rankings['new_world_ranking_times']!= 0) & (merged_rankings['world_rank_cwur']!= 0))|((merged_rankings['world_rank_cwur'] != 0)&(merged_rankings['new_world_ranking_sha']!= 0))| ((merged_rankings['new_world_ranking_times'] != 0)&(merged_rankings['new_world_ranking_sha']!= 0))]

merged_rankings_match

,university_name,new_world_ranking_sha,new_world_ranking_times,world_rank_cwur
1,northern arizona university,457.0,0.0,766.0
2,loyola university chicago,475.0,0.0,437.0
3,memorial university of newfoundland,484.0,553.0,611.0
5,hacettepe university,490.0,729.0,634.0
6,graduate university for advanced studies,498.0,0.0,524.0
...,...,...,...,...
1107,xidian university,0.0,777.0,991.0
1111,yeungnam university,0.0,701.0,770.0
1112,yokohama city university,0.0,617.0,513.0
1113,yokohama national university,0.0,737.0,518.0


In [85]:
merged_rankings_match['max_rank'] = merged_rankings_match[['new_world_ranking_sha','new_world_ranking_times','world_rank_cwur']].max(axis=1)

In [86]:
merged_rankings_match['min_rank'] = merged_rankings_match[merged_rankings_match[['new_world_ranking_sha','new_world_ranking_times','world_rank_cwur']] !=0].min(axis=1)

In [87]:
merged_rankings_match['diff_rank'] = merged_rankings_match['max_rank'] - merged_rankings_match['min_rank']

In [88]:
merged_rankings_match

,university_name,new_world_ranking_sha,new_world_ranking_times,world_rank_cwur,max_rank,min_rank,diff_rank
1,northern arizona university,457.0,0.0,766.0,766.0,457.0,309.0
2,loyola university chicago,475.0,0.0,437.0,475.0,437.0,38.0
3,memorial university of newfoundland,484.0,553.0,611.0,611.0,484.0,127.0
5,hacettepe university,490.0,729.0,634.0,729.0,490.0,239.0
6,graduate university for advanced studies,498.0,0.0,524.0,524.0,498.0,26.0
...,...,...,...,...,...,...,...
1107,xidian university,0.0,777.0,991.0,991.0,777.0,214.0
1111,yeungnam university,0.0,701.0,770.0,770.0,701.0,69.0
1112,yokohama city university,0.0,617.0,513.0,617.0,513.0,104.0
1113,yokohama national university,0.0,737.0,518.0,737.0,518.0,219.0


In [89]:
merged_rankings_match[merged_rankings_match['university_name']=='aarhus university']

,university_name,new_world_ranking_sha,new_world_ranking_times,world_rank_cwur,max_rank,min_rank,diff_rank
203,aarhus university,73.0,106.0,122.0,122.0,73.0,49.0


# **5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.**

In [90]:
times_most_recent

,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,new_total_score,group_index_times,new_world_ranking_times
0,2051,201-250,aalborg university,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15.0,48 : 52,2016,44.6,249,247
1,2059,251-300,aalto university,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17.0,32 : 68,2016,44.2,257,255
2,1908,106,aarhus university,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14.0,54 : 46,2016,57.7,106,106
3,2143,301-350,aberystwyth university,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18.0,48 : 52,2016,38.5,341,341
4,2555,601-800,adam mickiewicz university,Poland,20.0,25.7,11.0,15.3,28.7,-,"40,633",15.6,1.0,71 : 29,2016,16.5,753,753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,2540,601-800,yokohama national university,Japan,20.1,23.3,16.0,13.5,40.4,-,"10,117",12.1,8.0,28 : 72,2016,17.6,738,737
795,2150,301-350,yonsei university,South Korea,37.2,41.5,34.1,39.4,75.9,-,"24,774",11.6,14.0,NaN,2016,38.2,348,347
796,2149,301-350,york university,Canada,30.2,58.2,30.8,49.1,33.2,-,"48,007",39.4,9.0,59 : 41,2016,38.2,347,347
797,2458,601-800,yuan ze university,Taiwan,16.2,17.7,18.3,28.6,39.8,-,"8,663",20.6,4.0,43 : 57,2016,21.3,656,653


In [91]:
times_most_recent['fem_ratio'] = times_most_recent.female_male_ratio.str.extract(r'(.*):')
times_most_recent['male_ratio'] = times_most_recent.female_male_ratio.str.extract(r'.*:(.*)')

In [92]:
times_most_recent

,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,new_total_score,group_index_times,new_world_ranking_times,fem_ratio,male_ratio
0,2051,201-250,aalborg university,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15.0,48 : 52,2016,44.6,249,247,48,52
1,2059,251-300,aalto university,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17.0,32 : 68,2016,44.2,257,255,32,68
2,1908,106,aarhus university,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14.0,54 : 46,2016,57.7,106,106,54,46
3,2143,301-350,aberystwyth university,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18.0,48 : 52,2016,38.5,341,341,48,52
4,2555,601-800,adam mickiewicz university,Poland,20.0,25.7,11.0,15.3,28.7,-,"40,633",15.6,1.0,71 : 29,2016,16.5,753,753,71,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,2540,601-800,yokohama national university,Japan,20.1,23.3,16.0,13.5,40.4,-,"10,117",12.1,8.0,28 : 72,2016,17.6,738,737,28,72
795,2150,301-350,yonsei university,South Korea,37.2,41.5,34.1,39.4,75.9,-,"24,774",11.6,14.0,NaN,2016,38.2,348,347,NaN,NaN
796,2149,301-350,york university,Canada,30.2,58.2,30.8,49.1,33.2,-,"48,007",39.4,9.0,59 : 41,2016,38.2,347,347,59,41
797,2458,601-800,yuan ze university,Taiwan,16.2,17.7,18.3,28.6,39.8,-,"8,663",20.6,4.0,43 : 57,2016,21.3,656,653,43,57


In [93]:
times_most_recent = times_most_recent[times_most_recent['male_ratio'].notna()]

In [94]:
times_most_recent.dtypes

index                        int64
world_rank                  object
university_name             object
country                     object
teaching                   float64
international              float64
research                   float64
citations                  float64
income                     float64
total_score                 object
num_students                object
student_staff_ratio        float64
international_students     float64
female_male_ratio           object
year                         int64
new_total_score            float64
group_index_times            int64
new_world_ranking_times      int64
fem_ratio                   object
male_ratio                  object
dtype: object

In [95]:
times_most_recent['fem_ratio'] = times_most_recent['fem_ratio'].astype(int)
times_most_recent['male_ratio'] = times_most_recent['male_ratio'].astype(int)

In [96]:
times_most_recent['num_students'] = times_most_recent['num_students'].str.replace(',', '')
times_most_recent['num_students'] =times_most_recent['num_students'].astype(int)

In [97]:
times_most_recent['num_female'] = ( times_most_recent['fem_ratio']*times_most_recent['num_students']/100).round(0).astype(int)
times_most_recent['num_male'] = (times_most_recent['male_ratio']*times_most_recent['num_students']/100).round(0).astype(int)

In [98]:
times_most_recent

,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,...,international_students,female_male_ratio,year,new_total_score,group_index_times,new_world_ranking_times,fem_ratio,male_ratio,num_female,num_male
0,2051,201-250,aalborg university,Denmark,25.1,71.0,28.4,73.8,43.7,-,...,15.0,48 : 52,2016,44.6,249,247,48,52,8363,9059
1,2059,251-300,aalto university,Finland,31.1,65.4,32.8,62.1,61.6,-,...,17.0,32 : 68,2016,44.2,257,255,32,68,5152,10947
2,1908,106,aarhus university,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,...,14.0,54 : 46,2016,57.7,106,106,54,46,12903,10992
3,2143,301-350,aberystwyth university,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,...,18.0,48 : 52,2016,38.5,341,341,48,52,4441,4811
4,2555,601-800,adam mickiewicz university,Poland,20.0,25.7,11.0,15.3,28.7,-,...,1.0,71 : 29,2016,16.5,753,753,71,29,28849,11784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,2503,601-800,yeungnam university,South Korea,18.6,24.3,10.9,26.5,35.4,-,...,3.0,48 : 52,2016,19.5,701,701,48,52,10540,11418
794,2540,601-800,yokohama national university,Japan,20.1,23.3,16.0,13.5,40.4,-,...,8.0,28 : 72,2016,17.6,738,737,28,72,2833,7284
796,2149,301-350,york university,Canada,30.2,58.2,30.8,49.1,33.2,-,...,9.0,59 : 41,2016,38.2,347,347,59,41,28324,19683
797,2458,601-800,yuan ze university,Taiwan,16.2,17.7,18.3,28.6,39.8,-,...,4.0,43 : 57,2016,21.3,656,653,43,57,3725,4938


In [99]:
female_country = times_most_recent.groupby('country')['num_female'].sum()
male_country = times_most_recent.groupby('country')['num_male'].sum()
FM_Country = pd.merge(female_country, male_country, on=['country'])

In [100]:
FM_Country

,num_female,num_male
country,,
Argentina,67191,41182
Australia,353141,288564
Austria,64651,62400
Bangladesh,21323,41393
Belarus,20219,9084
Belgium,92920,76741
Brazil,247271,246980
Canada,344440,279188
Chile,24235,37509


# **6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)**

In [101]:
times_most_recent['ratio_']= times_most_recent['fem_ratio']/times_most_recent['male_ratio']

In [102]:
times_most_recent = times_most_recent[times_most_recent['male_ratio']!=0]

In [103]:
media = times_most_recent['ratio_'].mean()

In [104]:
ratio_inf=times_most_recent[times_most_recent['ratio_']<times_most_recent['ratio_'].mean()]

In [105]:
media

1.0858126447033292

In [106]:
len(ratio_inf)

374

In [107]:
ratio_inf

,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,...,female_male_ratio,year,new_total_score,group_index_times,new_world_ranking_times,fem_ratio,male_ratio,num_female,num_male,ratio_
0,2051,201-250,aalborg university,Denmark,25.1,71.0,28.4,73.8,43.7,-,...,48 : 52,2016,44.6,249,247,48,52,8363,9059,0.923077
1,2059,251-300,aalto university,Finland,31.1,65.4,32.8,62.1,61.6,-,...,32 : 68,2016,44.2,257,255,32,68,5152,10947,0.470588
3,2143,301-350,aberystwyth university,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,...,48 : 52,2016,38.5,341,341,48,52,4441,4811,0.923077
7,2510,601-800,ajou university,South Korea,19.5,20.0,11.9,23.9,45.7,-,...,33 : 67,2016,19.2,708,706,33,67,4193,8513,0.492537
8,2561,601-800,alexandria university,Egypt,20.3,33.7,8.2,14.1,29.7,-,...,46 : 54,2016,16.0,759,758,46,54,58618,68813,0.851852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,1814,12,yale university,United States of America,86.5,64.3,87.8,97.2,43.3,87.4,...,50 : 50,2016,87.4,12,12,50,50,5876,5876,1.000000
792,2503,601-800,yeungnam university,South Korea,18.6,24.3,10.9,26.5,35.4,-,...,48 : 52,2016,19.5,701,701,48,52,10540,11418,0.923077
794,2540,601-800,yokohama national university,Japan,20.1,23.3,16.0,13.5,40.4,-,...,28 : 72,2016,17.6,738,737,28,72,2833,7284,0.388889
797,2458,601-800,yuan ze university,Taiwan,16.2,17.7,18.3,28.6,39.8,-,...,43 : 57,2016,21.3,656,653,43,57,3725,4938,0.754386


# **7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country)**

In [108]:
ratio_inf_country = ratio_inf.groupby('country')['num_students'].sum().reset_index(name ='Sum_inf_avg_ratio')
ratio_inf_country.head() 

,country,Sum_inf_avg_ratio
0,Australia,160839
1,Austria,53607
2,Bangladesh,62716
3,Brazil,375248
4,Canada,86779


In [109]:
times_most_recent_grouped = times_most_recent.groupby('country')['num_students'].sum().reset_index(name ='Sum_total_country')
times_most_recent_grouped

,country,Sum_total_country
0,Argentina,108373
1,Australia,641705
2,Austria,127051
3,Bangladesh,62716
4,Belarus,29303
5,Belgium,169661
6,Brazil,494251
7,Canada,623629
8,Chile,61744
9,China,993461


In [110]:
merged = pd.merge(times_most_recent_grouped, ratio_inf_country, on= 'country')

In [112]:
merged['prop_inf_avg'] = merged['Sum_inf_avg_ratio']/merged['Sum_total_country']
merged.head()

,country,Sum_total_country,Sum_inf_avg_ratio,prop_inf_avg
0,Australia,641705,160839,0.250643
1,Austria,127051,53607,0.421933
2,Bangladesh,62716,62716,1.000000
3,Brazil,494251,375248,0.759226
4,Canada,623629,86779,0.139152


# **8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name**

In [113]:
attainment= pd.read_csv(r'C:\Users\valen\OneDrive\Documenti\Data science\Foundations of computer science\ProgettoCS\educational_attainment_supplementary_data.csv')

In [114]:
attainment.isnull()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,False,False,False,True,True,False,True,True,True,False,...,False,True,True,True,True,False,True,True,True,True
1,False,False,False,True,True,False,True,True,True,False,...,False,True,True,True,True,False,True,True,True,True
2,False,False,False,True,True,False,True,True,True,False,...,False,True,True,True,True,False,True,True,True,True
3,False,False,False,True,True,False,True,True,True,False,...,False,True,True,True,True,False,True,True,True,True
4,False,False,False,True,True,False,True,True,True,False,...,False,True,True,True,True,False,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79050,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
79051,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
79052,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
79053,False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [115]:
attainment = attainment[attainment['country_name'].notna()]

In [116]:
attainment = attainment[attainment['series_name'].notna()]

In [117]:
attainment[attainment['country_name'].isnull()]

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015


In [118]:
attainment[attainment['series_name'].isnull()]

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015


# **9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value**

In [119]:
attainment.head(5)

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


In [120]:
attainment_mod = attainment.melt(id_vars=["country_name", "series_name"], var_name="Year", value_name="Value")

In [121]:
attainment_mod.head(5)

,country_name,series_name,Year,Value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54


# **10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).**

In [122]:
shanghai_new = shanghai['university_name'].unique()

In [123]:
times_new = times['university_name'].unique()

In [124]:
cwur_new = cwur['university_name'].unique()

In [125]:
rank_list = [shanghai_new, cwur_new, times_new]  
concat_rankings = np.concatenate(rank_list)


In [126]:
occorrenze = collections.Counter(concat_rankings)
occorrenze

Counter({'harvard university': 3,
         'university of cambridge': 3,
         'stanford university': 3,
         'university of california berkeley': 3,
         'massachusetts institute of technology mit': 3,
         'california institute of technology': 3,
         'columbia university': 3,
         'princeton university': 3,
         'university of chicago': 3,
         'university of oxford': 3,
         'yale university': 3,
         'cornell university': 3,
         'university of california san diego': 3,
         'university of california los angeles': 3,
         'university of pennsylvania': 3,
         'university of wisconsin  madison': 1,
         'university of washington  seattle': 3,
         'university of california san francisco': 3,
         'the johns hopkins university': 3,
         'the university of tokyo': 1,
         'university of michigan  ann arbor': 1,
         'kyoto university': 3,
         'the imperial college of science technology and medicine': 

# **11. In the times ranking, compute the number of times each university appears**

In [127]:
rank_univ_times = times.groupby('university_name')['year'].count().reset_index(name = 'count')
rank_univ_times

,university_name,count
0,aalborg university,5
1,aalto university,5
2,aarhus university,6
3,aberystwyth university,5
4,adam mickiewicz university,1
...,...,...
794,yokohama national university,1
795,yonsei university,6
796,york university,5
797,yuan ze university,2


# **12. Find the universities that appear at most twice in the times ranking.**

In [128]:
rank_univ_times[rank_univ_times['count']<=2]

,university_name,count
4,adam mickiewicz university,1
5,agh university of science and technology,1
6,aix marseille university,1
7,ajou university,1
9,alexandru ioan cuza university,1
...,...,...
789,y ld z technical university,1
792,yeungnam university,1
793,yokohama city university,1
794,yokohama national university,1


# **13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).**

**Con procedimento alternativo proposto al punto 4 (ricalcolo dello score per attribuzione ranking)**

In [ ]:
shanghai_n = shanghai[['university_name', 'year', 'new_world_ranking_sha']]

In [ ]:
times_n = times[['university_name', 'year', 'new_world_ranking_times']]

In [ ]:
cwur_n = cwur[['university_name', 'year', 'world_rank_cwur']]

In [ ]:
merged_rankings_n = shanghai_n.merge(cwur_n,on=['university_name', 'year'], how = 'outer').merge(times_n,on=['university_name', 'year'], how='outer')[['university_name','year','new_world_ranking_sha','new_world_ranking_times','world_rank_cwur']]


In [ ]:
merged_rankings_n = merged_rankings_n.fillna(0)

In [ ]:
merged_rankings_match_n = merged_rankings_n[(merged_rankings_n['new_world_ranking_sha']!= 0)&(merged_rankings_n['new_world_ranking_times']!= 0)&(merged_rankings_n['world_rank_cwur'] != 0)]

In [ ]:
merged_rankings_match_n

In [ ]:
same_rank = merged_rankings_n[merged_rankings_n[['new_world_ranking_sha','new_world_ranking_times','world_rank_cwur']].nunique(axis=1) == 1]
same_rank